In [1]:
import pdfplumber


In [ ]:
with pdfplumber.open("Sample.pdf") as pdf:
    first_page = pdf.pages[1]
    text = first_page.extract_text()
    print(str(text))

MICHIGAN DEPARTMENT OF STATE
BUREAU OF ELECTIONS
96499
1. Committee I.D. Number ______________________________________________
SUMMARY PAGE
JUSTIN F. ROEBUCK FOR OTTAWA COUNTY CLERK/REGISTER
2. Committee Name _______________________________________________ _
CANDIDATE COMMITTEE
RECEIPTS Column I Column II
This Period Cumulative this election cycle
3. Contributions
1,999.18
a. Itemized (Schedule 1A - Column 6) (3a.) $ ___________________________
b. Unitemized (less than $20.01 each - no Schedule) (3b.) $ NOT APPLICABLE
1,999.18 119,870.76
c. Subtotal of "Contributions" (3c.) $ (18.) $ ____________________
0.00 0.00
4. Other Receipts (Schedule 1A -1, Column 6) (4.) $ __________________________ (19.) $ ____________________
1,999.18 119,870.76
5. TOTAL CONTRIBUTIONS AND OTHER RECEIPTS (5.) $ ___________________________ (20.) $ ____________________
(Add Line 3c + Line 4)
IN-KIND CONTRIBUTIONS & EXPENDITURES
56.13 329.69
6. In-Kind Contributions (Schedule 1-IK, Column 7) (6.) $ _____________

In [ ]:
import re

NUMBER_RE = re.compile(r'\(?-?\d[\d,]*(?:\.\d+)?\)?')

def totals_extraction(text, cont_ln=19, expnd_ln=35):
    lines = text.splitlines()
    if len(lines) < cont_ln:
        return None
    line = lines[cont_ln - 1]
    nums = NUMBER_RE.findall(line)
    if len(nums) < 2:
        return None
    cont_total = nums[1].strip('()').replace(',', '')
    line = lines[expnd_ln - 1]
    nums = NUMBER_RE.findall(line)
    if len(nums) < 2:
        return None
    expnd_total = nums[1].strip('()').replace(',', '')
    return cont_total, expnd_total

number = totals_extraction(text)
number

('119870.76', '41271.10')

In [3]:
with pdfplumber.open("Sample_Cover.pdf") as pdf:
    first_page = pdf.pages[0]
    text = first_page.extract_text()
    print(str(text))

FILED
MICHIGAN DEPARTMENT OF STATE
BUREAU OF ELECTIONS 25 JUL 2024 AM 11:57
CANDIDATE COMMITTEE OTTAWA COUNTY CLERK
FOR OFFICIAL USE ONLY
WEST OLIVE, MICHIGAN
COVER PAGE
R
th
e
e
p
t
o
re
rt
a
m
su
u
r
s
e
t
r
b
(o
e
r
l e
d
g
e
i
s
b
i
l
g
e
n
,
a
ty
te
p
d
e d
re
o
c
r
o
p
rd
ri n
k
t
e
e
e
d
p
i
e
n
r )
in
a
k
n
a
d
n
c
d
a
s
n
i
d
g
i
n
d
e
a
d
te
b
.
y 3.This Statement covers From:
01/01/2024 to 0 __ 7 _ / _ 2 _ 1 __ /2 __ 0 _ 2 __ 4 _________
1.C ommittee I.D. Number 4. Candidate Last Name First Name M.I.
96499 ROEBUCK JUSTIN F
4a. Office Sought Including District # or Community Served (If applicable)
2. Committee Name
CLERK/REGISTER, OTTAWA COUNTY
JUSTIN F. ROEBUCK FOR OTTAWA COUNTY CLERK/REGISTER
4b. County of Residence OTTAWA COUNTY
5.C ommittee's Mailing Address 6.Treasurer's Name & Residential Address
PO BOX 122 LAURA ROEBUCK
ZEELAND, MI 49464 27 WALL STREET
ZEELAND, MI 49464
(616) 799-1978
Ar ea Code and Phone
If the address in this box is different from the committee
maili

In [4]:
from bs4 import BeautifulSoup
import requests

base_url = "https://campaignfinance.us/ottawa.mi/"

committee_id = 96499
search_url = f"https://campaignfinance.us/ottawa.mi/iCommitteeSearchProcess.php?strSearchBy=1&iCommitteeVisibleID={committee_id}&cmdSubmit=Search"

print(search_url)

r = requests.get(search_url)
soup = BeautifulSoup(r.text, 'html.parser')
print(soup.prettify())



https://campaignfinance.us/ottawa.mi/iCommitteeSearchProcess.php?strSearchBy=1&iCommitteeVisibleID=96499&cmdSubmit=Search
<html>
 <head>
  <title>
   CampaignFinance.us - Ottawa County, Michigan
  </title>
  <link href="iCF.css" media="screen" rel="StyleSheet" type="text/css"/>
  <link href="style.css" rel="stylesheet"/>
  <link href="favicon.ico" rel="icon" type="image/x-ico"/>
  <link href="favicon.ico" rel="shortcut icon" type="image/x-icon"/>
 </head>
 <body>
  <table border="0" cellpadding="0" cellspacing="0" class="DataList">
   <tr>
    <td colspan="99">
     <table>
      <tr>
       <td>
        <img border="0" height="108px" name="controlbar7600" src="Ottawa_Final_CMYK_New_Icon_WhiteBackground.png" width="108px"/>
       </td>
       <td class="BigCounty">
        Ottawa County
       </td>
      </tr>
     </table>
     <form action="iCommitteeSearch.php" method="get" name="frmMe" onsubmit="">
      <input id="iCommitteeID" name="iCommitteeID" type="hidden" value=""/>
      

In [5]:
link_dict = {}
id_dict = {}

In [6]:
for link in soup.find_all('a'):
    x =  link.get('href')
    print(x)
    if x[:-3] == "iCommitteePortal.php?iCommitteeID=":
        print(x[:-3])
        link_dict[committee_id] = base_url + x
        id_dict[committee_id] = int(x[-3:])



iCommitteePortal.php?iCommitteeID=247
iCommitteePortal.php?iCommitteeID=
http://www.CampaignFinance.us


In [7]:
id_dict
link_dict

{96499: 'https://campaignfinance.us/ottawa.mi/iCommitteePortal.php?iCommitteeID=247'}

In [8]:
internal_id = id_dict[committee_id]
documents_url = f"https://campaignfinance.us/ottawa.mi/iDocuments.php?iCommitteeID={internal_id}&cmdOk=View+Campaign+Statements"
resp = requests.get(documents_url)
soup = BeautifulSoup(resp.text, 'html.parser')
print(soup.prettify())


<html>
 <head>
  <title>
   CampaignFinance.us - Ottawa County, Michigan
  </title>
  <link href="iCF.css" media="screen" rel="StyleSheet" type="text/css"/>
  <link href="style.css" rel="stylesheet"/>
  <link href="favicon.ico" rel="icon" type="image/x-ico"/>
  <link href="favicon.ico" rel="shortcut icon" type="image/x-icon"/>
 </head>
 <body>
  <table border="0" cellpadding="0" cellspacing="0">
   <tr>
    <td colspan="99">
     <table>
      <tr>
       <td>
        <img border="0" height="108px" name="controlbar7600" src="Ottawa_Final_CMYK_New_Icon_WhiteBackground.png" width="108px"/>
       </td>
       <td class="BigCounty">
        Ottawa County
       </td>
      </tr>
     </table>
     <form action="" method="get" name="frmMe" onsubmit="">
      <table class="DataList" width="618">
       <tr>
        <td colspan="100">
         <input id="cmdCancel" name="cmdCancel" onclick="history.back();" type="button" value="Return to Committee Details"/>
        </td>
       </tr>
      

In [9]:
for link in soup.find_all('a'):
    x =  link.get('href')
    print("https://campaignfinance.us" + x)

https://campaignfinance.us/MI-OTTAWA/IndexedDocs/6064.PDF
https://campaignfinance.us/MI-OTTAWA/IndexedDocs/5681.pdf
https://campaignfinance.us/MI-OTTAWA/IndexedDocs/5590.PDF
https://campaignfinance.us/MI-OTTAWA/IndexedDocs/3067.PDF
https://campaignfinance.us/MI-OTTAWA/IndexedDocs/3890.pdf
https://campaignfinance.us/MI-OTTAWA/IndexedDocs/3066.PDF
https://campaignfinance.us/MI-OTTAWA/IndexedDocs/1015.PDF
https://campaignfinance.us/MI-OTTAWA/IndexedDocs/1014.PDF
https://campaignfinance.us/MI-OTTAWA/IndexedDocs/1049.pdf
https://campaignfinance.us/MI-OTTAWA/IndexedDocs/1041.pdf
https://campaignfinance.us/MI-OTTAWA/IndexedDocs/1093.pdf
https://campaignfinance.us/MI-OTTAWA/IndexedDocs/1040.pdf
https://campaignfinance.us/MI-OTTAWA/IndexedDocs/1039.pdf
https://campaignfinance.us/MI-OTTAWA/IndexedDocs/1045.pdf
https://campaignfinance.us/MI-OTTAWA/IndexedDocs/1044.pdf
https://campaignfinance.us/MI-OTTAWA/IndexedDocs/1043.pdf
https://campaignfinance.us/MI-OTTAWA/IndexedDocs/1050.PDF
https://campai

In [10]:
url = "https://campaignfinance.us/MI-OTTAWA/IndexedDocs/6064.PDF"
output_path = "temp/6064.pdf"

response = requests.get(url, timeout=30)
response.raise_for_status()

with open(output_path, "wb") as f:
    f.write(response.content)

print("Downloaded", output_path)

Downloaded temp/6064.pdf


In [14]:
import pandas as pd

table = soup.find("table", class_="DataList")

records = []

for row in table.find_all("tr")[1:]:
    cells = row.find_all("td")
    if len(cells) < 5:
        continue

    link = cells[4].find("a")

    record = {
        "committee_id": committee_id,
        "document": cells[0].get_text(strip=True),
        "received": cells[1].get_text(strip=True),
        "due": cells[2].get_text(strip=True),
        "pages": cells[3].get_text(strip=True),
        "pdf_url": "https://campaignfinance.us" + link["href"] if link else None
    }
    if record["pdf_url"] == None:
        continue

    records.append(record)

df = pd.DataFrame(records)
pd.set_option("display.max_colwidth", None)

df.head()

,committee_id,document,received,due,pages,pdf_url
0,96499,Quarterly Stmt (Oct) - 2025,10/27/25,10/27/25,7,https://campaignfinance.us/MI-OTTAWA/IndexedDocs/6064.PDF
1,96499,Stmt of Organization (Amended),07/28/25,-,1,https://campaignfinance.us/MI-OTTAWA/IndexedDocs/5681.pdf
2,96499,Quarterly Stmt (Jul) - 2025,07/25/25,07/25/25,9,https://campaignfinance.us/MI-OTTAWA/IndexedDocs/5590.PDF
3,96499,Post-Election Stmt - 11/05/2024 General Election,12/03/24,12/05/24,4,https://campaignfinance.us/MI-OTTAWA/IndexedDocs/3067.PDF
4,96499,MiscellaneousCF COMPLIANCE AFFIDAVIT 11/5/24 POST ELECTION,11/26/24,-,1,https://campaignfinance.us/MI-OTTAWA/IndexedDocs/3890.pdf
